In [4]:
import ibis
from ibis.expr.tests.mocks import MockConnection

con = MockConnection()

Q1
---

    select
        l_returnflag,
        l_linestatus,
        sum(l_quantity) as sum_qty,
        sum(l_extendedprice) as sum_base_price,
        sum(l_extendedprice*(1-l_discount)) as sum_disc_price,
        sum(l_extendedprice*(1-l_discount)*(1+l_tax)) as sum_charge,
        avg(l_quantity) as avg_qty,
        avg(l_extendedprice) as avg_price,
        avg(l_discount) as avg_disc,
        count(*) as count_order
    from
        lineitem
    where
        l_shipdate <= date '1998-12-01' - interval '[DELTA]' day (3)
    group by
        l_returnflag,
        l_linestatus
    order by
        l_returnflag,
        l_linestatus;

Q2
---

"Minimum Cost Supplier Query"

    select
        s_acctbal,
        s_name,
        n_name,
        p_partkey,
        p_mfgr,
        s_address,
        s_phone,
        s_comment
    from
        part,
        supplier,
        partsupp,
        nation,
        region
    where
        p_partkey = ps_partkey
        and s_suppkey = ps_suppkey
        and p_size = [SIZE]
        and p_type like '%[TYPE]'
        and s_nationkey = n_nationkey
        and n_regionkey = r_regionkey
        and r_name = '[REGION]'
        and ps_supplycost = (
            select
            min(ps_supplycost)
            from
            partsupp, supplier,
            nation, region
            where
            p_partkey = ps_partkey
            and s_suppkey = ps_suppkey
            and s_nationkey = n_nationkey
            and n_regionkey = r_regionkey
            and r_name = '[REGION]'
        )

Values for substitution parameters:
1. SIZE = 15;
2. TYPE = BRASS;
3. REGION = EUROPE.

Q3
---

Shipping Priority Query

    select
        l_orderkey,
        sum(l_extendedprice*(1-l_discount)) as revenue,
        o_orderdate,
        o_shippriority
    from
        customer,
        orders,
        lineitem
    where
        c_mktsegment = '[SEGMENT]'
        and c_custkey = o_custkey
        and l_orderkey = o_orderkey
        and o_orderdate < date '[DATE]'
        and l_shipdate > date '[DATE]'
    group by
        l_orderkey,
        o_orderdate,
        o_shippriority
    order by
        revenue desc,
        o_orderdate;
        
Values for substitution parameters:
1. SEGMENT = BUILDING;
2. DATE = 1995-03-15.

Q4
---

Order Priority Checking Query

    select
        o_orderpriority,
        count(*) as order_count
    from orders
    where
        o_orderdate >= date '[DATE]'
        and o_orderdate < date '[DATE]' + interval '3' month
        and exists (
            select
            *
            from
                lineitem
            where
                l_orderkey = o_orderkey
            and l_commitdate < l_receiptdate
        )
    group by o_orderpriority
    order by o_orderpriority;
    
Values for substitution parameters:
1. DATE = 1993-07-01

Q8
---

National Market Share Query

    select o_year,
    sum(case
        when nation = '[NATION]'
        then volume
        else 0
        end) / sum(volume) as mkt_share
    from (
        select
        extract(year from o_orderdate) as o_year,
        l_extendedprice * (1-l_discount) as volume,
        n2.n_name as nation
        from
            part,
            supplier,
            lineitem,
            orders,
            customer,
            nation n1,
            nation n2,
            region
        where
            p_partkey = l_partkey
            and s_suppkey = l_suppkey
            and l_orderkey = o_orderkey
            and o_custkey = c_custkey
            and c_nationkey = n1.n_nationkey
            and n1.n_regionkey = r_regionkey
            and r_name = '[REGION]'
            and s_nationkey = n2.n_nationkey
            and o_orderdate between date '1995-01-01' and date '1996-12-31'
            and p_type = '[TYPE]'
    ) as all_nations
    group by o_year
    order by o_year;
    
Values for substitution parameters:
1. NATION = BRAZIL;
2. REGION = AMERICA;
3. TYPE = ECONOMY ANODIZED STEEL.

Q17
---

Small-Quantity-Order Revenue Query

    select sum(l_extendedprice) / 7.0 as avg_yearly
    from
        lineitem,
        part
    where
        p_partkey = l_partkey
        and p_brand = '[BRAND]'
        and p_container = '[CONTAINER]'
        and l_quantity < (
            select
            0.2 * avg(l_quantity)
            from lineitem
            where l_partkey = p_partkey
        );

Values for substitution parameters:
1. BRAND = Brand#23;
2. CONTAINER = MED BOX.

Q20
---

    SELECT   s_name, 
             s_address 
    FROM     supplier, 
             nation 
    WHERE    s_suppkey IN 
             ( 
                    SELECT ps_suppkey 
                    FROM   partsupp 
                    WHERE  ps_partkey IN 
                           ( 
                                  SELECT p_partkey 
                                  FROM   part 
                                  WHERE  p_name LIKE '[COLOR]%' ) 
                    AND    ps_availqty > 
                           ( 
                                  SELECT 0.5 * SUM(l_quantity) 
                                  FROM   lineitem 
                                  WHERE  l_partkey = ps_partkey 
                                  AND    l_suppkey = ps_suppkey 
                                  AND    l_shipdate >= DATE('[DATE]’) and l_shipdate < date('[DATE]’) + interval ‘1’ year ) ) 
    AND      s_nationkey = n_nationkey 
    AND      n_name = '[NATION]' 
    ORDER BY s_name;

Q21
---

    SELECT s_name, 
           Count(*) AS numwait 
    FROM   supplier, 
           lineitem l1, 
           orders, 
           nation 
    WHERE  s_suppkey = l1.l_suppkey 
           AND o_orderkey = l1.l_orderkey 
           AND o_orderstatus = 'F' 
           AND l1.l_receiptdate > l1.l_commitdate 
           AND EXISTS (SELECT * 
                       FROM   lineitem l2 
                       WHERE  l2.l_orderkey = l1.l_orderkey 
                              AND l2.l_suppkey <> l1.l_suppkey) 
           AND NOT EXISTS (SELECT * 
                           FROM   lineitem l3 
                           WHERE  l3.l_orderkey = l1.l_orderkey 
                                  AND l3.l_suppkey <> l1.l_suppkey 
                                  AND l3.l_receiptdate > l3.l_commitdate) 
           AND s_nationkey = n_nationkey 
           AND n_name = '[NATION]' 
    GROUP  BY s_name 
    ORDER  BY numwait DESC, 
              s_name; 

1. NATION = SAUDI ARABIA.

Q22
---

    SELECT 
        cntrycode, 
        Count(*) AS numcust, 
        Sum(c_acctbal) AS totacctbal 
    FROM ( 
        SELECT 
        substring(c_phone from 1 FOR 2) AS cntrycode, 
        c_acctbal 
    FROM 
        customer 
    WHERE 
        substring(c_phone FROM 1 FOR 2) IN 
        ('[I1]','[I2]’,'[I3]','[I4]','[I5]','[I6]','[I7]') 
        and c_acctbal > ( 
            select avg(c_acctbal) 
            from customer 
            where c_acctbal > 0.00 and 
                    substr (c_phone from 1 for 2) in ('[I1]','[I2]','[I3]','[I4]','[I5]','[I6]','[I7]') ) 
                    and not exists ( 
                    select * 
                    from orders 
                    where o_custkey = c_custkey ) ) 
        as custsale 
        group by cntrycode 
        order by cntrycode;

1. I1 = 13.
2. I2 = 31.
3. I3 = 23.
4. I4 = 29.
5. I5 = 30.
6. I6 = 18.
7. I7 = 17.